In [4]:
from src.preprocessing.preprocess_ucr import UCRDatasetImporter
from src.preprocessing.data_pipeline import build_data_pipeline
from src.utils import load_yaml_param_settings

from src.experiments.tester import RepTester
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from src.models.vqvae_representations import PretrainedVQVAE
from src.experiments.classnet_test import train_ClassNet

In [7]:
config_dir = 'src/configs/config.yaml' #dir to config file

config = load_yaml_param_settings(config_dir)

# data pipeline
dataset_importer = UCRDatasetImporter(**config['dataset'])
batch_size = config['dataset']['batch_sizes']['vqvae']
train_data_loader, test_data_loader = [build_data_pipeline(batch_size, dataset_importer, config, kind) for kind in ['train', 'test']]

input_length = train_data_loader.dataset.X.shape[-1]

#Data
X_train = train_data_loader.dataset.X
Y_train = train_data_loader.dataset.Y

X_test = test_data_loader.dataset.X
Y_test = test_data_loader.dataset.Y

#z_q and their indeces. Also the codebook
base = PretrainedVQVAE(input_length, config)
z_q_train, ind_train = base.run_through_encoder_codebook(train_data_loader)
z_q_test, ind_test = base.run_through_encoder_codebook(test_data_loader)
codebook = base.vq_model.codebook

self.X_train.shape: (30, 128)
self.X_test.shape: (900, 128)
# unique labels (train): [0 1 2]
# unique labels (test): [0 1 2]
encoder-CBF.ckpt loaded..
decoder-CBF.ckpt loaded..
vq_model-CBF.ckpt loaded..


In [6]:
zqs_train = z_q_train.view(z_q_train.size(0), -1)

classfier = train_ClassNet(zqs_train, Y_train)

torch.Size([30, 2560])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (30x2560 and 30x2560)